<a href="https://colab.research.google.com/github/Alihassan7726/Language-Models/blob/main/Word_level_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### I've cleaned and transformed the text in seperate file " Word level language model using GloVe" . I'll be importing the same cleaned data to build a model .  

In [ ]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = '/content/gdrive/MyDrive/Colab DataSets/socrates_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
lines[:3]

['book i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was',
 'i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted',
 'i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted with']

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [ ]:
word_index=tokenizer.word_index
vocab_size = len(word_index)+1
print('Number of unique words:',vocab_size)

Number of unique words: 7410


In [ ]:
from pickle import dump
# saving the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [ ]:
# separate into input and output
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

# Splitting the Data
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)
del(X)
del(y)

Train shape: (106769, 50) Val shape: (11864, 50)


In [ ]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *


In [ ]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 250, input_length=seq_length))
model.add(Bidirectional(LSTM(250,return_sequences=True)))
model.add(LSTM(250))
model.add(Dense(250, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

print(model.summary())
#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy']) 

#Adding callbacks
mc=ModelCheckpoint('/content/gdrive/MyDrive/Colab Notebooks/best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True,verbose=1)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 250)           1852500   
_________________________________________________________________
bidirectional (Bidirectional (None, 50, 500)           1002000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 250)               751000    
_________________________________________________________________
dense (Dense)                (None, 250)               62750     
_________________________________________________________________
dense_1 (Dense)              (None, 7410)              1859910   
Total params: 5,528,160
Trainable params: 5,528,160
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from time import time
t = time()
history = model.fit(X_tr , y_tr , batch_size=100 , epochs=50,
                    validation_data=( X_val , y_val ),verbose=1,callbacks=[mc],
                    )
print("Total time taken to run : {} mins".format(np.round((time()-t)/60,decimals = 2)))


Epoch 1/50
1068/1068 [==============================] - 66s 62ms/step - loss: 5.9655 - accuracy: 0.0906 - val_loss: 5.6587 - val_accuracy: 0.1166

Epoch 00001: val_accuracy improved from -inf to 0.11657, saving model to /content/gdrive/MyDrive/Colab Notebooks/best_model.h5
Epoch 2/50
1068/1068 [==============================] - 59s 56ms/step - loss: 5.4212 - accuracy: 0.1364 - val_loss: 5.5053 - val_accuracy: 0.1428

Epoch 00002: val_accuracy improved from 0.11657 to 0.14278, saving model to /content/gdrive/MyDrive/Colab Notebooks/best_model.h5
Epoch 3/50
1068/1068 [==============================] - 60s 56ms/step - loss: 5.1652 - accuracy: 0.1573 - val_loss: 5.4423 - val_accuracy: 0.1537

Epoch 00003: val_accuracy improved from 0.14278 to 0.15366, saving model to /content/gdrive/MyDrive/Colab Notebooks/best_model.h5
Epoch 4/50
1068/1068 [==============================] - 59s 56ms/step - loss: 4.9819 - accuracy: 0.1691 - val_loss: 5.4427 - val_accuracy: 0.1565

Epoch 00004: val_accuracy

In [ ]:
## Saving current model instead of using previous model
model.save('Final_model_socorates.h5')

In [ ]:
# load doc into memory for testing
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = '/content/gdrive/MyDrive/Colab DataSets/socrates_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')


In [ ]:
seq_length = len(lines[0].split()) - 1

In [ ]:
from random import randint
from pickle import load
from keras.models import load_model
# load the model
model_2 = load_model('Final_model_socorates.h5')
# load the tokenizer
tokenizer_2 = load(open('tokenizer.pkl', 'rb'))

# Selecting a random seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

that from the individual they pass into the state how else can they come there take the quality of passion or spirit it would be ridiculous to imagine that this quality when found in states is not derived from the individuals who are supposed to possess it eg the thracians scythians



In [ ]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [ ]:
# generate new text
generated = generate_seq(model_2, tokenizer_2, seq_length, seed_text, 50)
print(generated)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


and and goes blinking for the offices of which the oligarchy is so incredulous and virtue and which every value has enumerated among the oaks of the rocks and cherisher of every sort of knowledge but vice and children are their brothers he said nor are they mind about the


In [ ]:
print(seed_text)

that from the individual they pass into the state how else can they come there take the quality of passion or spirit it would be ridiculous to imagine that this quality when found in states is not derived from the individuals who are supposed to possess it eg the thracians scythians


In [ ]:
generated = generate_seq(model_2, tokenizer_2, seq_length, seed_text, 1)
print(generated)

and


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
## Another example
seed_text = lines[randint(0,len(lines))]
generated = generate_seq(model_2, tokenizer_2, seq_length, seed_text, 50)
print(generated)
print(seed_text , '...' , generated)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


they are the counterpart of men of health still supplying faculties many and fair to them that will be his meaning he said then socrates in this stage of the slights which you propose before in this particular state there are tyrannies and this is said to have similar qualities
or subjectmatter of knowledge and knowledge is to know the nature of being yes and opinion is to have an opinion yes and do we know what we opine or is the subjectmatter of opinion the same as the subjectmatter of knowledge nay he replied that has been already disproven if ... they are the counterpart of men of health still supplying faculties many and fair to them that will be his meaning he said then socrates in this stage of the slights which you propose before in this particular state there are tyrannies and this is said to have similar qualities
